In [ ]:
import json
import pandas as pd
from pathlib import Path

# Thư mục chứa các file .jsonl
input_folder = Path(r"D:\MangXaHoi\output_rows1")   
# Thư mục xuất file CSV
output_folder = Path(r"D:\MangXaHoi\gameshow_vietnam\file_csv1")
output_folder.mkdir(parents=True, exist_ok=True)

# Duyệt tất cả file .jsonl trong thư mục
for jsonl_file in input_folder.glob("*.jsonl"):
    records = []
    with open(jsonl_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
                records.append(obj)
            except Exception as e:
                print(f"Lỗi parse dòng trong {jsonl_file}: {e}")

    if not records:
        print(f"[WARN] File {jsonl_file} không có dữ liệu.")
        continue

    df = pd.DataFrame(records)

    # Xuất file CSV với cùng tên vào thư mục file_csv
    csv_file = output_folder / (jsonl_file.stem + ".csv")
    df.to_csv(csv_file, index=False, encoding="utf-8-sig")
    print(f"[OK] {jsonl_file.name} → {csv_file}")


In [13]:
import re
import pandas as pd
from pathlib import Path
import json

# Thư mục chứa các file CSV gốc
INPUT_DIR = Path(r"D:/MangXaHoi/gameshow_vietnam/file_csv1")
# Thư mục lưu file CSV tên nghệ sĩ tương ứng
OUTPUT_DIR = Path(r"D:/MangXaHoi/gameshow_vietnam/artists_csv1")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Regex nhận diện tên tiếng Việt
VI_NAME_RE = re.compile(
    r"\b([A-ZÀ-ỴĐ][a-zà-ỹđ]+(?:\s+[A-ZÀ-ỴĐ][a-zà-ỹđ]+){1,4})\b"
)

def normalize(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def extract_names_from_text(text: str):
    """Trích tất cả tên trong 1 ô"""
    text = normalize(str(text))
    text = re.sub(r"[;/•|]", ",", text)  # chuẩn hóa phân tách
    names = set()
    for m in VI_NAME_RE.finditer(text):
        names.add(normalize(m.group(1)))
    return names

def pick_name_columns(df: pd.DataFrame):
    """Chọn cột khả năng là tên"""
    NAME_HINTS = ["Tên", "thành viên", "nghệ sĩ", "artist", "member", "cast", "Khách mời", "Thành viên", "Anh trai", "Tên nghệ sĩ", 
    "Em xinh", "MC", "Trưởng Phòng", "Khách mời 1" ,"Khách mời 2" ,"Khách mời 3", "Khách mời 4", "Huấn luyện viên", "Giám khảo", "Thí sinh", "Chỉ huy"]
    cols = [c for c in df.columns if any(kw in str(c).lower() for kw in NAME_HINTS)]
    if cols:
        return cols
    # fallback: chọn cột có nhiều ô khớp regex tên nhất
    scores = {}
    for c in df.columns:
        cnt = sum(bool(extract_names_from_text(v)) for v in df[c].astype(str).head(40))
        scores[c] = cnt
    if scores:
        best = max(scores, key=scores.get)
        return [best]
    return []

# Xử lý từng file
for csv_file in INPUT_DIR.glob("*.csv"):
    try:
        df = pd.read_csv(csv_file, encoding="utf-8-sig")
    except:
        df = pd.read_csv(csv_file, encoding="utf-8")

    name_cols = pick_name_columns(df)
    artist_names = set()

    if name_cols:
        for col in name_cols:
            for val in df[col]:
                artist_names.update(extract_names_from_text(val))
        print(f"[{csv_file.name}] → Cột tên: {name_cols}")
    else:
        # fallback: quét toàn bộ bảng
        for col in df.columns:
            for val in df[col]:
                artist_names.update(extract_names_from_text(val))
        print(f"[{csv_file.name}] → Không rõ cột tên, quét toàn bảng.")

    # Làm sạch, sắp xếp
    cleaned = sorted(n for n in artist_names if len(n.split()) >= 2)

    # Lưu ra CSV tương ứng
    out_file = OUTPUT_DIR / (csv_file.stem + "_artists.csv")
    pd.DataFrame({"artist_name": cleaned}).to_csv(out_file, index=False, encoding="utf-8-sig")
    print(f"[OK] Lưu {len(cleaned)} tên vào {out_file}")


[Bí_mật_đêm_Chủ_Nhật_table_01.csv] → Cột tên: ['Người chơi 1']
[OK] Lưu 9 tên vào D:\MangXaHoi\gameshow_vietnam\artists_csv1\Bí_mật_đêm_Chủ_Nhật_table_01_artists.csv
[Bí_mật_đêm_Chủ_Nhật_table_02.csv] → Cột tên: ['Người chơi 1']
[OK] Lưu 8 tên vào D:\MangXaHoi\gameshow_vietnam\artists_csv1\Bí_mật_đêm_Chủ_Nhật_table_02_artists.csv
[Bí_mật_đêm_Chủ_Nhật_table_03.csv] → Cột tên: ['Người chơi 1']
[OK] Lưu 7 tên vào D:\MangXaHoi\gameshow_vietnam\artists_csv1\Bí_mật_đêm_Chủ_Nhật_table_03_artists.csv
[Bước_nhảy_hoàn_vũ_table_01.csv] → Cột tên: ['Cúp bạc']
[OK] Lưu 8 tên vào D:\MangXaHoi\gameshow_vietnam\artists_csv1\Bước_nhảy_hoàn_vũ_table_01_artists.csv
[Bước_nhảy_hoàn_vũ_table_02.csv] → Cột tên: ['Người dẫn']
[OK] Lưu 10 tên vào D:\MangXaHoi\gameshow_vietnam\artists_csv1\Bước_nhảy_hoàn_vũ_table_02_artists.csv
[Bước_nhảy_hoàn_vũ_table_03.csv] → Cột tên: ['Giám khảo']
[OK] Lưu 22 tên vào D:\MangXaHoi\gameshow_vietnam\artists_csv1\Bước_nhảy_hoàn_vũ_table_03_artists.csv
[Bước_nhảy_hoàn_vũ_table_

In [14]:
import pandas as pd
from pathlib import Path
import re

# === cấu hình ===
INPUT_DIR = Path(r"D:/MangXaHoi/gameshow_vietnam/artists_csv1")   # chứa các file *_artists.csv
OUTPUT_WIDE = Path(r"D:/MangXaHoi/gameshow_vietnam/all_shows_artists_wide1.csv")
OUTPUT_LONG = Path(r"D:/MangXaHoi/gameshow_vietnam/all_shows_artists_long1.csv")

rows = []
long_records = []

for f in sorted(INPUT_DIR.glob("*.csv")):
    # Lấy tên file, bỏ phần _table... và _artists
    show_name = f.stem
    show_name = re.sub(r"_table.*", "", show_name)   # bỏ từ _table trở đi
    show_name = show_name.replace("_artists", "")    # bỏ hậu tố _artists

    try:
        df = pd.read_csv(f, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(f, encoding="utf-8")

    col = "artist_name" if "artist_name" in df.columns else df.columns[0]
    artists = sorted(set(str(x).strip() for x in df[col].dropna() if str(x).strip()))

    # Dạng dài
    for name in artists:
        long_records.append({"show": show_name, "artist_name": name})

    # Dạng rộng
    row = {"show": show_name}
    for i, name in enumerate(artists, start=1):
        row[f"artist_{i}"] = name
    rows.append(row)

# Xuất dạng rộng
wide_df = pd.DataFrame(rows)
artist_cols = [c for c in wide_df.columns if c.startswith("artist_")]
artist_cols = sorted(artist_cols, key=lambda x: int(x.split("_")[1]))
wide_df = wide_df[["show"] + artist_cols]
wide_df.to_csv(OUTPUT_WIDE, index=False, encoding="utf-8-sig")
print(f"[OK] Xuất dạng rộng → {OUTPUT_WIDE}")

# Xuất dạng dài
if long_records:
    long_df = pd.DataFrame(long_records).sort_values(["show", "artist_name"])
    long_df.to_csv(OUTPUT_LONG, index=False, encoding="utf-8-sig")
    print(f"[OK] Xuất dạng dài → {OUTPUT_LONG}")


[OK] Xuất dạng rộng → D:\MangXaHoi\gameshow_vietnam\all_shows_artists_wide1.csv
[OK] Xuất dạng dài → D:\MangXaHoi\gameshow_vietnam\all_shows_artists_long1.csv


In [15]:
import pandas as pd

# File wide
file_path = r"D:/MangXaHoi/gameshow_vietnam/all_shows_artists_wide1.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# Lấy tất cả cột nghệ sĩ
artist_cols = [c for c in df.columns if c.startswith("artist_")]

# Gom tất cả tên nghệ sĩ thành một list
all_artists = []
for col in artist_cols:
    all_artists.extend(df[col].dropna().astype(str).str.strip().tolist())

# Bỏ trùng, bỏ rỗng, sắp xếp
unique_artists = sorted(set(a for a in all_artists if a and a.lower() != "nan"))

# Lưu thành CSV 1 cột
out_file = r"D:/MangXaHoi/gameshow_vietnam/all_unique_artists1.csv"
pd.DataFrame({"artist_name": unique_artists}).to_csv(out_file, index=False, encoding="utf-8-sig")

print(f"Đã lấy {len(unique_artists)} nghệ sĩ duy nhất → {out_file}")


Đã lấy 982 nghệ sĩ duy nhất → D:/MangXaHoi/gameshow_vietnam/all_unique_artists1.csv


In [16]:
import pandas as pd

df = pd.read_csv(r"D:/MangXaHoi/gameshow_vietnam/all_unique_artists1.csv", encoding="utf-8-sig")

duplicates = df[df.duplicated(subset=["artist_name"], keep=False)]

if not duplicates.empty:
    print("Có nghệ sĩ bị trùng:")
    print(duplicates)
else:
    print("Không có nghệ sĩ nào bị trùng trong file all_unique_artists.csv")


Không có nghệ sĩ nào bị trùng trong file all_unique_artists.csv


In [ ]:
# -*- coding: utf-8 -*-
import re
import time
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote

# ================== CẤU HÌNH ==================
INPUT_ARTISTS_CSV = r"D:/MangXaHoi/gameshow_vietnam/all_unique_artists1.csv"  # cột 'artist_name'
OUTPUT_CSV        = r"D:/MangXaHoi/gameshow_vietnam/wiki_infobox_artists1.csv"
LANG              = "vi"   # 'vi' là Wikipedia tiếng Việt
SLEEP_BETWEEN     = 0.8    # nghỉ giữa các request để lịch sự
TIMEOUT           = 20
# ==============================================

WIKI_BASE = f"https://{LANG}.wikipedia.org/wiki/"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ArtistInfoboxScraper/1.0; +for-academic-use)"
}

def normalize_page_title(name: str) -> str:
    """
    Đưa 'Trấn Thành' -> 'Trấn_Thành' rồi URL-encode.
    Giữ nguyên dấu tiếng Việt, Wikipedia hiểu.
    """ 
    name = (name or "").strip()
    title = name.replace(" ", "_")
    # URL-encode toàn bộ chuỗi (giữ _), để xử lý dấu tiếng Việt đúng
    return quote(title, safe="_")

def fetch_html(url: str) -> str:
    r = requests.get(url, headers=HEADERS, timeout=TIMEOUT)
    if r.status_code == 200:
        return r.text
    # Nếu 404, thử tìm bằng API opensearch để lấy trang gần nhất
    if r.status_code == 404:
        q = url.split("/wiki/")[-1]
        search_q = re.sub(r"_", " ", q)
        api = f"https://{LANG}.wikipedia.org/w/api.php"
        params = {
            "action": "opensearch",
            "search": search_q,
            "limit": 1,
            "namespace": 0,
            "format": "json"
        }
        rs = requests.get(api, params=params, headers=HEADERS, timeout=TIMEOUT)
        if rs.status_code == 200:
            data = rs.json()
            if len(data) >= 4 and data[3]:
                candidate_url = data[3][0]
                rr = requests.get(candidate_url, headers=HEADERS, timeout=TIMEOUT)
                if rr.status_code == 200:
                    return rr.text
    r.raise_for_status()

def clean_text(s: str) -> str:
    s = re.sub(r"\s+", " ", (s or "").strip())
    return s

def parse_infoboxes(html: str) -> dict:
    """
    Trả về dict các trường từ tất cả infobox trên trang (nếu có nhiều, sẽ merge).
    Ưu tiên lấy text trong th/td; nếu có nhiều tr, gộp bằng dấu ' | '.
    """
    soup = BeautifulSoup(html, "html.parser")
    infoboxes = soup.select('[class*="infobox"]')
    result = {}

    for box in infoboxes:
        # Lấy tiêu đề infobox (thường là tên trang)
        caption = box.find("caption")
        if caption:
            result.setdefault("infobox_caption", clean_text(caption.get_text(" ", strip=True)))

        # Duyệt các hàng
        for tr in box.select("tr"):
            # trường hợp cả th và td
            th = tr.find("th")
            td = tr.find("td")

            if th and td:
                key = clean_text(th.get_text(" ", strip=True))
                val = clean_text(td.get_text(" ", strip=True))
                # Chuẩn hóa key (ví dụ đổi về snake_case nhẹ)
                key_norm = re.sub(r"\s+", "_", key.lower())
                # Gộp nếu trùng key
                if key_norm in result and val:
                    result[key_norm] = f"{result[key_norm]} | {val}"
                else:
                    result[key_norm] = val
            # Nếu chỉ có th (dạng header), có thể bỏ qua hoặc lưu riêng
            # elif th and not td:
            #     ...

    return result

def main():
    # 1) Đọc danh sách nghệ sĩ
    artists_df = pd.read_csv(INPUT_ARTISTS_CSV, encoding="utf-8-sig")
    name_col = "artist_name" if "artist_name" in artists_df.columns else artists_df.columns[0]
    artists = [str(x).strip() for x in artists_df[name_col].dropna() if str(x).strip()]

    all_rows = []

    for i, name in enumerate(artists, start=1):
        page_title = normalize_page_title(name)
        url = WIKI_BASE + page_title
        try:
            html = fetch_html(url)
        except Exception as e:
            print(f"[{i}/{len(artists)}] {name}: LỖI tải trang {url} → {e}")
            all_rows.append({"artist_name": name, "wiki_url": url, "error": str(e)})
            time.sleep(SLEEP_BETWEEN)
            continue

        try:
            info = parse_infoboxes(html)
            row = {"artist_name": name, "wiki_url": url}
            # gộp các trường infobox vào dòng
            for k, v in info.items():
                row[k] = v
            all_rows.append(row)
            print(f"[{i}/{len(artists)}] {name}: OK ({len(info)} fields)")
        except Exception as e:
            print(f"[{i}/{len(artists)}] {name}: LỖI parse infobox → {e}")
            all_rows.append({"artist_name": name, "wiki_url": url, "error": f"parse: {e}"})

        time.sleep(SLEEP_BETWEEN)

    # 3) Lưu CSV
    out_df = pd.DataFrame(all_rows)

    # === Thêm ID dựa vào link Wikipedia ===
    import hashlib
    def generate_id(url):
        if pd.isna(url):
            return None
        return hashlib.md5(url.encode('utf-8')).hexdigest()[:8]  # 8 ký tự đầu

    out_df["artist_id"] = out_df["wiki_url"].apply(generate_id)

    # Sắp cột: artist_id, artist_name, wiki_url, infobox_caption, ...
    cols = list(out_df.columns)
    fixed = [c for c in ["artist_id", "artist_name", "wiki_url", "infobox_caption"] if c in cols]
    others = [c for c in cols if c not in fixed]
    out_df = out_df[fixed + others]

    out_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print(f"Đã lưu {len(out_df)} nghệ sĩ → {OUTPUT_CSV}")


if __name__ == "__main__":
    main()


[1/982] Adam Levine: OK (9 fields)
[2/982] Aleksandar Iliev Vachev: LỖI tải trang https://vi.wikipedia.org/wiki/Aleksandar_Iliev_Vachev → 404 Client Error: Not Found for url: https://vi.wikipedia.org/wiki/Aleksandar_Iliev_Vachev
[3/982] Ali Hoàng Dương: LỖI tải trang https://vi.wikipedia.org/wiki/Ali_Ho%C3%A0ng_D%C6%B0%C6%A1ng → 404 Client Error: Not Found for url: https://vi.wikipedia.org/wiki/Ali_Ho%C3%A0ng_D%C6%B0%C6%A1ng
[4/982] Ali Hoàng Dương Thanh Lam: LỖI tải trang https://vi.wikipedia.org/wiki/Ali_Ho%C3%A0ng_D%C6%B0%C6%A1ng_Thanh_Lam → 404 Client Error: Not Found for url: https://vi.wikipedia.org/wiki/Ali_Ho%C3%A0ng_D%C6%B0%C6%A1ng_Thanh_Lam
[5/982] Andrea Bocelli: OK (6 fields)
[6/982] Angela Phương Trinh: OK (4 fields)
[7/982] Angela Phương Trinh Kristian Yordanov: LỖI tải trang https://vi.wikipedia.org/wiki/Angela_Ph%C6%B0%C6%A1ng_Trinh_Kristian_Yordanov → 404 Client Error: Not Found for url: https://vi.wikipedia.org/wiki/Angela_Ph%C6%B0%C6%A1ng_Trinh_Kristian_Yordanov
[8/9

In [18]:
import pandas as pd

# File đầu vào
INPUT = r"D:/MangXaHoi/gameshow_vietnam/wiki_infobox_artists1.csv"
WITH_DATA = r"D:/MangXaHoi/gameshow_vietnam/artists_with_data1.csv"
NO_DATA   = r"D:/MangXaHoi/gameshow_vietnam/artists_no_data1.csv"

# Đọc file
df = pd.read_csv(INPUT, encoding="utf-8-sig")

# Lấy tất cả cột xuất hiện sau wiki_url
if "wiki_url" in df.columns:
    cols_after_url = df.columns[df.columns.get_loc("error")+1:]
else:
    raise ValueError("Không tìm thấy cột 'wiki_url' trong file.")

# Chia nhóm
with_data = df[df[cols_after_url].notna().any(axis=1)]   # có ít nhất 1 dữ liệu
no_data   = df[~df.index.isin(with_data.index)]          # toàn bộ trống

# Xuất file
with_data.to_csv(WITH_DATA, index=False, encoding="utf-8-sig")
no_data.to_csv(NO_DATA, index=False, encoding="utf-8-sig")

print(f"Có dữ liệu: {len(with_data)} nghệ sĩ → {WITH_DATA}")
print(f"Không có dữ liệu: {len(no_data)} nghệ sĩ → {NO_DATA}")


Có dữ liệu: 458 nghệ sĩ → D:/MangXaHoi/gameshow_vietnam/artists_with_data1.csv
Không có dữ liệu: 524 nghệ sĩ → D:/MangXaHoi/gameshow_vietnam/artists_no_data1.csv


In [21]:
import pandas as pd

# Đường dẫn 2 file CSV
file1 = r"D:/MangXaHoi/gameshow_vietnam/all_shows_artists_long1.csv"
file2 = r"D:/MangXaHoi/gameshow_vietnam/all_shows_artists_long.csv"

# Đọc dữ liệu
df1 = pd.read_csv(file1, encoding="utf-8-sig")
df2 = pd.read_csv(file2, encoding="utf-8-sig")

# Gộp lại
merged = pd.concat([df1, df2], ignore_index=True)

# Lưu ra file mới
output = r"D:/MangXaHoi/gameshow_vietnam/merge_all_shows_artists_long.csv"
merged.to_csv(output, index=False, encoding="utf-8-sig")

print(f"Đã gộp {len(df1)} + {len(df2)} = {len(merged)} dòng vào: {output}")


Đã gộp 1395 + 1301 = 2696 dòng vào: D:/MangXaHoi/gameshow_vietnam/merge_all_shows_artists_long.csv


In [24]:
# Đọc file all_shows_artists_long.csv để làm ground truth
long_file = r"D:/MangXaHoi/gameshow_vietnam/merge_all_shows_artists_long.csv"
long_df = pd.read_csv(long_file, encoding="utf-8-sig")

# Tạo ground truth: mỗi nghệ sĩ gán với gameshow đầu tiên mà họ tham gia
ground_truth = {}
for artist, group in long_df.groupby("artist_name"):
    shows = group["show"].tolist()
    if shows:
        ground_truth[artist] = shows[0]   # lấy gameshow đầu tiên làm nhãn


In [25]:
import pandas as pd

df = pd.read_csv(r"D:/MangXaHoi/gameshow_vietnam/merge_artists_with_data.csv", encoding="utf-8-sig")

duplicates = df[df.duplicated(subset=["artist_name"], keep=False)]

if not duplicates.empty:
    print("Có nghệ sĩ bị trùng:")
    print(duplicates)
else:
    print("Không có nghệ sĩ nào bị trùng trong file all_unique_artists.csv")


Có nghệ sĩ bị trùng:
    artist_id          artist_name  \
0    572d005d  Angela Phương Trinh   
2    09ba692c              Anh Thư   
3    6759faeb              Anh Đức   
13   74af5de5          Bích Phương   
16   b283a223            Bạch Long   
..        ...                  ...   
808  ca156656           Đỗ Duy Nam   
811  c3ba25c2             Đức Hùng   
812  43070099             Đức Phúc   
813  c4f1cb42       Ưng Hoàng Phúc   
814  897526cc         Ốc Thanh Vân   

                                              wiki_url infobox_caption  \
0    https://vi.wikipedia.org/wiki/Angela_Ph%C6%B0%...             NaN   
2           https://vi.wikipedia.org/wiki/Anh_Th%C6%B0             NaN   
3    https://vi.wikipedia.org/wiki/Anh_%C4%90%E1%BB...             NaN   
13   https://vi.wikipedia.org/wiki/B%C3%ADch_Ph%C6%...             NaN   
16     https://vi.wikipedia.org/wiki/B%E1%BA%A1ch_Long             NaN   
..                                                 ...             ...   
808 

In [26]:
import pandas as pd

# ===== Đường dẫn =====
WITH_ARTISTS = r"D:/MangXaHoi/gameshow_vietnam/merge_artists_with_data.csv"      # cột: artist_name
LONG_FILE    = r"D:/MangXaHoi/gameshow_vietnam/merge_all_shows_artists_long.csv" # cột: show, artist_name
OUT_MATRIX   = r"D:/MangXaHoi/gameshow_vietnam/artist_coappearance_matrix1.csv"
# =====================

# Đọc dữ liệu
with_df = pd.read_csv(WITH_ARTISTS, encoding="utf-8-sig")
long_df = pd.read_csv(LONG_FILE, encoding="utf-8-sig")

# Chuẩn hoá
with_df["artist_name"] = with_df["artist_name"].astype(str).str.strip()
long_df["artist_name"] = long_df["artist_name"].astype(str).str.strip()
long_df["show"]        = long_df["show"].astype(str).str.strip()

# Danh sách nghệ sĩ cần xét
artists = with_df["artist_name"].dropna().unique().tolist()

# Giữ lại trong long_df chỉ các nghệ sĩ nằm trong danh sách này
long_df = long_df[long_df["artist_name"].isin(artists)].drop_duplicates(["show","artist_name"])

# Tạo ma trận xuất hiện (nghệ sĩ x show)
M = pd.crosstab(long_df["artist_name"], long_df["show"]).astype(int)

# Đảm bảo đủ tất cả nghệ sĩ từ with_df (nếu ai chưa xuất hiện show nào thì thêm dòng 0)
M = M.reindex(artists, fill_value=0)

# Tính ma trận đồng xuất hiện: C = M · Mᵀ
C = M.dot(M.T)

# Bỏ giá trị chéo (tự ghép)
for i in range(len(C)):
    C.iat[i, i] = 0

# Xuất CSV
C.to_csv(OUT_MATRIX, encoding="utf-8-sig")

print(f"Đã tạo ma trận đồng xuất hiện {C.shape[0]} x {C.shape[1]} tại: {OUT_MATRIX}")


Đã tạo ma trận đồng xuất hiện 675 x 675 tại: D:/MangXaHoi/gameshow_vietnam/artist_coappearance_matrix1.csv
